# Train
training model

In [1]:
from train import initialize_globals, Hyperparameters, pipline

initialize_globals()
hyperparameters = Hyperparameters(
    epochs=50,
    early_stop_patience=5,
    lr_warmup_epochs=5,
    lr=0.0001
)

hyperparameters

mps is available device


{'model': 'stn_mobilenet_small',
 'val_ratio': 0.2,
 'data_ratio': 1,
 'batch_size': 64,
 'epochs': 50,
 'early_stop_patience': 5,
 'target_val_acc': 100.0,
 'opt_name': 'sgd',
 'momentum': 0.9,
 'lr': 0.0001,
 'lr_scheduler_name': 'cosine',
 'lr_warmup_epochs': 5,
 'lr_warmup_method': 'linear',
 'lr_warmup_decay': 0.01,
 'weight_decay': 2e-05,
 'norm_weight_decay': 0.0,
 'label_smoothing': 0.1,
 'train_crop_size': 176,
 'val_crop_size': 224}

In [2]:
pipline(hyperparameters)

wandb: Currently logged in as: eason. Use `wandb login --relogin` to force relogin


splitting dataset...


Splitting 遊戲行為 data: 100%|██████████| 1772/1772 [00:01<00:00, 1560.40it/s]


Loading training data...
Loading validation data...
Creating data loaders...
Creating model...
Creating criterion...
Creating optimizer...
Creating learning rate scheduler...
Start training...


Epoch 0:   0%|          | 0/465 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1605ad3a0>
Traceback (most recent call last):
  File "/Users/Ethan/Developer/Projects/Intern/glowing/國美展覽/arty/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/Ethan/Developer/Projects/Intern/glowing/國美展覽/arty/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Py

Error occurred: <class 'NotImplementedError'>, The operator 'aten::hardsigmoid.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS., <traceback object at 0x16692cf40>


NotImplementedError: The operator 'aten::hardsigmoid.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.